In [1]:
import numpy as np

In [2]:
def initialize():
    global T, J, JR, γ, egam, β, α, δ, tol, damp, itermax, w, r, wn, Rn, p, K, \
    L, A, C, I, B, G, k, U, Y, Ba, Bf, h, Tb, TXr, Tpen, eps, n_p, gy, by, κ, lsra_on, \
    smopec, τk, τc, τw, τr, τp, τs, tax, pen, a, c, v, m, ψ, beq, ω_b, GAM, BQ, pam


    T = 24; J = 3; JR = 3; γ = .5; egam = 1.0 - 1.0/γ; β = .9
    α = .3; δ = .0; tol = 1e-6; damp = 0.2; itermax = 200


    w  = np.zeros(T+1); r = np.zeros(T+1); wn = np.zeros(T+1)
    Rn = np.zeros((J,T+1)); p = np.zeros(T+1); K  = np.zeros(T+1)
    L  = np.zeros(T+1); A = np.zeros(T+1); C  = np.zeros(T+1)
    I  = np.zeros(T+1); B = np.zeros(T+1); G  = np.zeros(T+1)
    k  = np.zeros(T+1); U = np.zeros((J,T+1)); Y = np.zeros(T+1)
    Ba = np.zeros(T+1); Bf = np.zeros(T+1); h = np.zeros(J)
    Tb = np.zeros(T+1); TXr = np.zeros(T+1); Tpen = np.zeros(T+1)
    eps = np.zeros(T+1); ψ = np.zeros((J,T+1))
    beq = np.zeros((J,T+1)); ω_b = np.zeros(J); GAM = np.zeros((J,T+1))
    BQ = np.zeros(T+1)


    n_p = np.zeros(T+1) + .2    
    gy  = 0.2
    by  = np.zeros(T+1)
    κ   = np.zeros(T+1)
    lsra_on = False
    smopec = False
    pam = True
    
    ψ[0,:] = 1.0
    ψ[1,:] = 0.85
    ψ[2,:] = 0.8


    τk  = np.zeros(T+1)
    τc = np.zeros(T+1)
    τw = np.zeros(T+1)
    τr = np.zeros(T+1)
    τp = np.zeros(T+1)
    τs = np.zeros(T+1)
    tax = np.ones(T+1)
    pen = np.zeros((J,T+1))


    a = np.zeros((J,T+1))
    c = np.zeros((J,T+1))
    v = np.zeros(T+J-1)
    
    h[0:JR-1] = 1.0
    h[JR-1:J] = .0
    
    ω_b[0] = .5
    ω_b[1] = .5
    ω_b[2] = .0


    m = np.zeros((J,T+1))
    for t in range(T+1):
        m[0,t] = 1.0
        tm = year(t, 1, 0)
        for j in range(1,J):
            m[j,t] = m[j-1,tm]*ψ[j,t]/(1.0 + n_p[t])
            
        GAM_total = ω_b[0]
        for j in range(1,J):
            GAM_total = GAM_total + ω_b[j]*m[j,t]
            
        for j in range(J):
            GAM[j,t] = ω_b[j]/GAM_total
    return None


In [3]:
def year(t, j, jp):
    year = t + jp - j
    if t == 0 or year<=0:
        year = 0
    if t == T or year>=T:
        year = T
    return year

In [4]:
def factor_prices(t):
    global k, r, w, wn, Rn, p
    k[t]   = K[t]/L[t]                
    if smopec and t > 0:
        r[t] = r[0]
    else:
        r[t] = (1.0 - τk[t])/(1.0 - eps[t]*τk[t])*(α*k[t]**(α-1.0)-δ)
  
    w[t]   = (1.0 - α)*k[t]**α

    wn[t]  = w[t]*(1.0 - τw[t] - τp[t])
    p[t]   = 1.0 + τc[t]
    
    for j in range(J):
        if pam and t>=2:
            Rn[j,t]  = (1.0 + r[t]*(1.0 - τr[t]))/ψ[j,t]
        else:
            Rn[j,t]  = 1.0 + r[t]*(1.0 - τr[t])  
    
    
    return None

In [5]:
def get_W(j, t):
    
    Assets = wn[t]*h[j] + beq[j,t] + pen[j,t]
    
    if t == 1 and j > 0:
        Assets = Assets + Rn[j,t] * a[j,t] + v[J - j - 1] ## calful about index!!!
    if t >= 1 and j == 0:
        Assets = Assets + v[t+J-2]
        
    PRn = 1.0
    for jp in range(j+1,J):
        tp = year(t,j,jp)
        PRn = PRn*Rn[jp,tp]
        Assets = Assets + (wn[tp]*h[jp] + beq[jp,tp] + pen[jp,tp])/PRn
    return Assets


def get_Psi(j,t):
    Psi = 1.0
    PRn = 1.0
    PPs = 1.0
    
    for jp in range(j+1,J):
        tp = year(t, j ,jp)
        PRn = PRn*Rn[jp,tp]
        PPs = PPs*ψ[jp,tp]
        Psi = Psi + (β**(jp-j)*PPs)**γ*(p[tp]/(PRn*p[t]))**(1.0-γ)
    Psi = 1.0/(p[t]*Psi)
    
    return Psi


In [6]:
def get_path(j,t):
    
    global c, a, beq
    
    PRn = 1.0
    beq[j,t] = GAM[j,t]*BQ[t]

    for jp in range(j+1,J):
        
        tp = year(t,j,jp)
        tm = year(t,j,jp-1)
        PRn = PRn*ψ[jp,tp]*Rn[jp,tp]
        
        c[jp,tp] = (β**(jp-j)*PRn*p[t]/p[tp])**γ*c[j,t]
        beq[jp,tp] = GAM[jp,tp]*BQ[tp]
        
        a[jp,tp] = wn[tm]*h[jp-1] + beq[jp-1,tm] + pen[jp-1,tm] + Rn[jp-1,tm]*a[jp-1,tm] - p[tm]*c[jp-1,tm]
        if tp == 2:
            a[jp,tp] = a[jp,tp] + v[J-jp]
        if tp > 2 and jp == 1:
            a[jp,tp] = a[jp,tp] + v[tm+J-2]
    return None


In [7]:
def decisions(t):
    global c
    
    c[0,t] = get_Psi(0,t)*get_W(0,t)
    get_path(0,t)
    
    if t == 1:
        for j in range(1,J):
            c[j,t] = get_Psi(j,t)*get_W(j,t)
            get_path(j,t)
    return None

In [8]:
## calculating quantities in a certain year

def quantities(t):
    global G, C, A, L, Y, B, K, Bf, Tb, I, BQ
    
    tm = year(t,1,0)
    tp = year(t,0,1)
    
    if t == 0:
        G[t] = gy*Y[t]
    else:
        G[t] = G[0]
        
    # aggregate individual decisions
    C[t] = .0
    A[t] = .0
    L[t] = .0
    BQ[t] = .0
    
    for j in range(J):
        C[t] = C[t] + c[j,t]*m[j,t]
        A[t] = A[t] + a[j,t]*m[j,t]/ψ[j,t]
        L[t] = L[t] + h[j]*m[j,t]
        BQ[t] = BQ[t] + ((1.0+r[t]*(1.0-τr[t]))/ψ[j,t] - Rn[j,t])*a[j,t]*m[j,t]
    
    Y[t] = K[t]**α*L[t]**(1.0-α)
    B[t] = by[tm]*Y[t]
    
    # derive capital in small open or closed economy
    if smopec and t>0:
        K[t]  = L[t]*((r[t]*(1.0-eps[t]*τk[t])/(1.0-τk[t]) + δ)/α)**(1.0/(α-1.0)) 
        Bf[t] = A[t] - K[t] - Ba[t] - B[t]
        Tb[t] = (1.0+n_p[tp])*Bf[tp] - (1.0+r[t])*Bf[t]
    else:
        K[t] = damp*(A[t]-B[t]-Ba[t]) + (1.0-damp)*K[t]
    
    I[t] = (1.0+n_p[tp])*K[tp] - (1.0 - δ)*K[t]
    
    return None


In [9]:
# calculating government parameters

def government(t):
    global τc, τk, τw, τr, pen, Tpen, τp, TXr
    tp = year(t, 0, 1)
    
    taxrev = np.zeros(4)
    
    taxrev[0] = τc[t]*C[t]
    taxrev[1] = τw[t]*w[t]*L[t]
    taxrev[2] = τr[t]*r[t]*A[t]
    taxrev[3] = τk[t]*(Y[t] - w[t]*L[t]-(δ+eps[t]*r[t])*K[t])
    
    
    if tax[t] == 1:      
        τc[t] = ((1.0 + r[t])*B[t] + G[t] - (taxrev[1] + taxrev[2] + \
                    taxrev[3]+(1.0 + n_p[tp])*B[tp]))/C[t]   
    elif tax[t] == 2:   
        τw[t] = ((1.0 + r[t])*B[t] + G[t] - (taxrev[0] + taxrev[3] + \
                    (1.0+n_p[tp])*B[tp]))/(w[t]*L[t] + r[t]*A[t])
        τr[t] = τw[t]    
    elif tax[t] == 3:    
        τw[t] = ((1.0 + r[t])*B[t] + G[t] - (taxrev[0] + taxrev[2] + \
                    taxrev[3] + (1.0 + n_p[tp])*B[tp]))/(w[t]*L[t])
    else:               
        τr[t] = ((1.0 + r[t])*B[t] + G[t] - (taxrev[0] + taxrev[1] + \
                    taxrev[3]+ (1.0 + n_p[tp])*B[tp]))/(r[t]*A[t])
    
    TXr[t] = np.sum(taxrev)
    
    # get budget balancing social security contribution
    pen[JR-1:J,t] = κ[t]*w[t]
    Tpen[t] = .0
    
    for j in range(JR-1,J):
        Tpen[t] = Tpen[t] + pen[j,t]*m[j,t]
        
    τp[t] = Tpen[t]/(w[t]*L[t])
    return None

In [10]:
# compute lide expectancy of household who enters in year t
def ET(t):
    ET = .0
    PPs = 1.0
    
    for j in range(J):
        tp = year(t, 0, j)
        tp1 = year(t, 0, j+1)
        PPs = PPs*ψ[j, tp]
    
        if j<J-1:
            ET = ET + (j+1)*PPs*(1.0-ψ[j+1, tp1])
        else:
            ET = ET + (j+1)*PPs
    return ET

In [11]:
## Compute household utility

def utility(t):
    global U
    
    # for first generation
    U[0,t] = .0
    PPs = 1.0
    for j in range(J):
        tp = year(t, 0, j)
        PPs = PPs*ψ[j,tp]
        U[0,t] = U[0,t] + β**j*PPs*c[j,tp]**egam/egam
    
    # for current total population if year = 0 or 1
    if t < 2:
        for j in range(1,J):
            U[j,t] = c[j,t]**egam/egam
            PPs = 1.0
            for jp in range(j+1,J):
                tp = year(t, j, jp)
                PPs = PPs*ψ[jp,tp]
                U[j,t] = U[j,t] + β**(jp-j)*PPs*c[jp,tp]**egam/egam
                
    return None

In [12]:
## Calculating lsra transfers

def lsra():
    global Ba, v, ustar

    for t in range(1,T+1):
        utility(t)   
        
    Ba = np.zeros(T+1)
    for j in range(1,J):
        v[J-j-1] = v[J-j-1] + get_W(j,1)*((U[j,0]/U[j,1])**(1.0/egam)-1.0)
        Ba[2] = Ba[2] + v[J-j-1]*m[j,1]
    
    PV = v[T+J-2]*(1.0+r[T])/(r[T]-n_p[T])
    sum1 = get_W(0,T)*(1.0+r[T])/(r[T]-n_p[T])
    sum2 = get_W(0,T)*(U[0,T]*egam)**(-1.0/egam)*(1.0+r[T])/(r[T]-n_p[T])
    
    for t in range(T-1,0,-1):
        tp = year(t,0,1)
        PV = PV*(1.0+n_p[tp])/(1.0+r[tp]) + v[t+J-2]
        sum1 = sum1*(1.0+n_p[tp])/(1.0+r[tp]) + get_W(0,t)
        sum2 = sum2*(1.0+n_p[tp])/(1.0+r[tp]) + get_W(0,t)*(U[0,t]*egam)**(-1.0/egam)
        
    ustar = ((sum1 - Ba[2] - PV)/sum2)**egam/egam
    
    for t in range(1,T+1):
        v[t+J-2] = v[t+J-2] + get_W(0,t)*((ustar/U[0,t])**(1.0/egam)-1.0)
        if t == 2:
            Ba[2] = (Ba[2] + v[J-1])/(1.0 + n_p[2])
        if t > 2:
            Ba[t] = ((1.0+r[t-1])*Ba[t-1] + v[J+t-3])/(1.0+n_p[t])
    
    return None

In [13]:
def get_SteadyState():
    K[0] = 1.0
    L[0] = 1.0
    for i in range(itermax+1):
        factor_prices(0)
        decisions(0)
        quantities(0)
        government(0)
        
        if abs(Y[0]-C[0]-I[0]-G[0])/Y[0] < tol:
            break
    utility(0)
    
    if i < itermax:
        print('Iteration: ', i, ' Diff: ',abs(Y[0]-C[0]-I[0]-G[0])/Y[0])
    else:
        print('!!! No equilibrium found !!!')
    
    return None

In [14]:
## sovles for transition path using Gauss-Seidel:

def get_Transition():
    global a, K, L, HEV
    # Initialize values from initial equilibrium
    a[:,1] = a[:,0]
    K[:] = K[0]
    L[:] = L[0]
    
    nmarket = 0
    
    for ite in range(itermax+2):
        # get prices, decisions and quantites
        
        for t in range(1,T+1):
            factor_prices(t)
        
        for t in range(1,T+1):
            decisions(t)
        
        if lsra_on:
            lsra()
        
        for t in range(1,T+1):
            quantities(t)
        
        for t in range(1,T+1):
            government(t)
            
        # check for the number of markets in equilibrium
        nmarket = 0
        for t in range(1,T+1):
            if  abs(Y[t]-C[t]-I[t]-G[t]-Tb[t])/Y[t] < tol:
                nmarket = nmarket + 1
        if nmarket == T:
            break
        #print('Iteration: ',ite,' Markets: ',nmarket,' Diff: ',np.max(np.abs(Y-C-I-G-Tb)/Y))
    if ite < itermax:
        print('Iteration: ',ite,' Markets: ',nmarket,' Diff: ',np.max(np.abs(Y-C-I-G-Tb)/Y))
    else:
        print('!!! No equilibrium found !!!')
        
        
    for t in range(1,T+1):
        utility(t)
    HEV = np.zeros(T+2)
    for t in range(2,T+2):
        HEV[t] = ((U[0,t-1]/U[0,0])**(1/egam)-1)*100
    HEV[1] = ((U[1,1]/U[1,0])**(1/egam)-1)*100 # generation 0
    HEV[0] = ((U[2,1]/U[2,0])**(1/egam)-1)*100 # generation -1
        
    return None

In [15]:
# Table 7.13
initialize()
get_SteadyState()
get_Transition()
print('-----------------------------------------------')
print('Table 7.13 Introducing pay-as-you-go pensions\nwith uncertain life span')
print('-----------------------------------------------')
print(' t   τc    C   BQ    K    Y    w    r   HEV')
print('-----------------------------------------------')
print(-1,'                                   ','%2.2f'%HEV[0])
list1 = [0, 1, 2, 3, 4]
for t in list1:
    print('%2.f'%t,'%.2f'%τc[t],'%.2f'%C[t],'%.2f'%BQ[t],'%.2f'%K[t],'%.2f'%Y[t],'%.2f'%w[t],'%.2f'%r[t],'%2.2f'%HEV[t+1])
print(' .   .    .    .    .    .    .    .    .')
print(' .   .    .    .    .    .    .    .    .')
print(' .   .    .    .    .    .    .    .    .')
for t in [T]:
    print('%2.f'%t,'%.2f'%τc[t],'%.2f'%C[t],'%.2f'%BQ[t],'%.2f'%K[t],'%.2f'%Y[t],'%.2f'%w[t],'%.2f'%r[t],'%2.2f'%HEV[t+1])
print('-----------------------------------------------')


Iteration:  83  Diff:  9.3664779348104e-07
Iteration:  72  Markets:  24  Diff:  9.445100034513017e-07
-----------------------------------------------
Table 7.13 Introducing pay-as-you-go pensions
with uncertain life span
-----------------------------------------------
 t   τc    C   BQ    K    Y    w    r   HEV
-----------------------------------------------
-1                                     0.13
 0 0.27 0.71 0.09 0.24 0.94 0.39 1.20 8.66
 1 0.27 0.71 0.09 0.24 0.94 0.39 1.20 3.75
 2 0.26 0.73 0.00 0.23 0.94 0.38 1.21 -6.06
 3 0.27 0.70 0.00 0.21 0.92 0.38 1.28 -8.30
 4 0.28 0.67 0.00 0.20 0.90 0.37 1.35 -10.26
 .   .    .    .    .    .    .    .    .
 .   .    .    .    .    .    .    .    .
 .   .    .    .    .    .    .    .    .
24 0.29 0.66 0.00 0.19 0.88 0.36 1.40 -11.69
-----------------------------------------------
